## Quora 2

Trying to match vocab to word embeddings - not working currently

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

C:\Users\richard\Anaconda3\envs\tf15\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [7]:
train_df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [8]:
## split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)
## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

In [9]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [10]:
sentences = train_df["question_text"].progress_apply(lambda x: x.split()).values

100%|████████████████████████████████████████████████████████████████████| 1175509/1175509 [00:03<00:00, 386605.68it/s]


In [11]:
sentences

array([list(['What', 'have', 'been', 'the', 'best', 'exhibits', 'at', 'the', 'Museo', 'del', 'Prado', 'in', 'Madrid?']),
       list(['How', 'can', 'I', 'rotate', 'batch', 'image', 'files?']),
       list(['Which', 'is', 'the', 'best', 'cable', 'operator', 'in', 'Thane', 'west', 'area?']),
       ...,
       list(['Do', 'we', 'need', 'a', 'prescription', 'for', 'cough', 'syrup', 'in', 'Egypt?']),
       list(['What', 'are', 'the', 'best', 'and', 'worst', 'aspects', 'of', 'being', 'a', 'travel', 'agent?']),
       list(['Who', 'was', 'a', 'person', 'you', 'met', 'who', 'gave', 'a', 'very', 'good', 'vibe/', 'good-spirit', 'that', 'you', 'remained', 'friends', 'with', 'through', 'life?'])],
      dtype=object)

In [12]:
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

100%|████████████████████████████████████████████████████████████████████| 1175509/1175509 [00:02<00:00, 442285.65it/s]


{'What': 375815, 'have': 74917, 'been': 12323, 'the': 588148, 'best': 55168}


In [13]:
from gensim.models import KeyedVectors

news_path = 'D:\\ml_code\\embeddings\\GoogleNews-vectors-negative300\\GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(news_path, binary=True)

C:\Users\richard\Anaconda3\envs\tf15\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [29]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [30]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████████████████████████████████████████████████████████████████| 508819/508819 [00:00<00:00, 613188.59it/s]


Found embeddings for 24.31% of vocab
Found embeddings for  85.88% of all text


In [31]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [32]:
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_text(x))
sentences = train["question_text"].apply(lambda x: x.split())
vocab = build_vocab(sentences)

100%|████████████████████████████████████████████████████████████████████| 1306122/1306122 [00:03<00:00, 391127.20it/s]


In [33]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [34]:
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_numbers(x))
sentences = train["question_text"].progress_apply(lambda x: x.split())
vocab = build_vocab(sentences)

100%|████████████████████████████████████████████████████████████████████| 1306122/1306122 [00:03<00:00, 420642.90it/s]


In [35]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [36]:
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
sentences = train["question_text"].progress_apply(lambda x: x.split())
to_remove = ['a','to','of','and']
sentences = [[word for word in sentence if not word in to_remove] for sentence in tqdm(sentences)]
vocab = build_vocab(sentences)

100%|████████████████████████████████████████████████████████████████████| 1306122/1306122 [00:03<00:00, 413118.53it/s]


In [37]:
train_df.head()

,qid,question_text,target
651064,7f8590ef60e30b4344fd,What have been the best exhibits at the Museo ...,0
1294259,fda9538a2e0a5b2dfc3c,How can I rotate batch image files,0
1205706,ec528b4e3abc3347cd21,Which is the best cable operator in Thane west...,0
460090,5a1a41ea2086f2264eab,How do I expand factor and simplify in algebra,0
1277213,fa4f394af94b2b094e15,Do you judge people often,0


In [39]:
train_df['question_text'][:100]

651064     What have been the best exhibits at the Museo ...
1294259                   How can I rotate batch image files
1205706    Which is the best cable operator in Thane west...
460090        How do I expand factor and simplify in algebra
1277213                            Do you judge people often
700336     How do I get above ## in ICSE ##th in all subj...
312154                          How do I use the grid method
977640     Can you hang a poster diagonally if you have a...
648670     Do many Bulgarians consider Macedonia part of ...
168063     Did you have any superb business India on your...
47318      How is the current condition of journalism or ...
858766     How would you write a short story involving Fo...
1238649    How can I delete my question in the Partner Pr...
59860                                     What is ## of ####
117013     Would Steve Jobs still be successful if he wer...
930095     What are the best books for HTML Bootstrap Jquery
1213698    Is it possibl

In [40]:
del embeddings_index
import gc; gc.collect()
time.sleep(10)

In [43]:
## fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

In [46]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))

In [47]:
train_X = tokenizer.texts_to_sequences(train_X)

In [48]:
#repeat for the val and test
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences to the same length
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [49]:
#load the entire GloVe word embedding file into memory as a dictionary of
#word to embedding array
path = 'D:\\ml_code\\embeddings\\glove.840B.300d\\'
EMBEDDING_FILE = f'{path}glove.840B.300d.txt'

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE,errors='ignore'))

In [50]:
#create a matrix of one embedding for each word in the training
#dataset. We can do that by enumerating all unique words in the Tokenizer.word index and
#locating the embedding weight vector from the loaded GloVe embedding. The result is a matrix
#of weights only for words we will see during training.

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

#initialise the embedding matrix
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

#for each word in vocab get weights
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [51]:
#prepare a first simple model
# define model
model = Sequential()
input_length = Input(shape=(maxlen,))
e = Embedding(max_features, embed_size, weights=[embedding_matrix], input_length=maxlen)
model.add(e)
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
#compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 30001     
Total params: 15,030,001
Trainable params: 15,030,001
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
#run for just one epoch
model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y),verbose=2)

Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
 - 83s - loss: 0.9964 - acc: 0.9378 - val_loss: 1.0027 - val_acc: 0.9378


In [53]:
#apply to validation set
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)

130613/130613 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 4us/step


In [54]:
#look for a better threshold
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.0
F1 score at threshold 0.11 is 0.0
F1 score at threshold 0.12 is 0.0
F1 score at threshold 0.13 is 0.0
F1 score at threshold 0.14 is 0.0
F1 score at threshold 0.15 is 0.0
F1 score at threshold 0.16 is 0.0
F1 score at threshold 0.17 is 0.0
F1 score at threshold 0.18 is 0.0
F1 score at threshold 0.19 is 0.0
F1 score at threshold 0.2 is 0.0
F1 score at threshold 0.21 is 0.0
F1 score at threshold 0.22 is 0.0
F1 score at threshold 0.23 is 0.0
F1 score at threshold 0.24 is 0.0
F1 score at threshold 0.25 is 0.0
F1 score at threshold 0.26 is 0.0
F1 score at threshold 0.27 is 0.0
F1 score at threshold 0.28 is 0.0
F1 score at threshold 0.29 is 0.0
F1 score at threshold 0.3 is 0.0
F1 score at threshold 0.31 is 0.0
F1 score at threshold 0.32 is 0.0
F1 score at threshold 0.33 is 0.0
F1 score at threshold 0.34 is 0.0
F1 score at threshold 0.35 is 0.0
F1 score at threshold 0.36 is 0.0
F1 score at threshold 0.37 is 0.0
F1 score at threshold 0.38 is 0.0
F1 score at thres

In [55]:
#do a clean up
del word_index, embeddings_index, all_embs, embedding_matrix, model
import gc; gc.collect()
time.sleep(10)